In [14]:
import numpy
import unittest
import math
import cv2
from functools import partial

#1.1a) This function is pretty easy to test. As it only does maths, we can simply test, if the returned values are correct.
def bilinear_interpolate( p00, p01, p10, p11, x, y):
    
    first_interpolationX1 = p10 * x + p00 * (1-x)
    first_interpolationX2 = p01 * x + p11 * (1-x)
    second_interpolation = first_interpolationX2 * y +first_interpolationX1 * (1-y)
    
    return second_interpolation

#1.2a) As we don't twist the image, i will use the corners of the new image, to calculate its new size.
#1.2b) I'm using numpy.linalg.inv, so i need to test with a function, whose first parameter actually is a 2D-array, wich is not
#a singluar matrix. The correct functionality of the function can only be tested with matrixes, whose effects are known. To test
#the correct application of transform_function, the tests for the single transformations can be used. This also tests the
#resizing of the new canvas and correct placement of the new pixels. For testing transformations where the canvas keeps its
#size, an extended canvas can be used.
def transform_image(image, transform_function):
    height, width, channels = image.shape
    
    corners = [(0, 0), (0, height - 1), (width - 1, 0), (width - 1, height - 1)]
    transformed_corners = [transform_function(x, y) for x, y in corners]
    
    min_x = min(point[0] for point in transformed_corners)
    max_x = max(point[0] for point in transformed_corners)
    min_y = min(point[1] for point in transformed_corners)
    max_y = max(point[1] for point in transformed_corners)
    
    #min_x_old = min(point[0] for point in corners)
    #max_x_old = max(point[0] for point in corners)
    #min_y_old = min(point[1] for point in corners)
    #max_y_old = max(point[1] for point in corners)
    #min_x = min(min_x, min_x_old)
    #max_x = max(max_x, max_x_old)
    #min_y = min(min_y, min_y_old)
    #max_y = max(max_y, max_y_old)
    
    new_width = int(max_x - min_x + 1)
    new_height = int(max_y - min_y + 1)
    
    if(transform_function.args[0][2,0] ==0):
        new_width = width
        new_height = height
    
    result = numpy.zeros((new_height, new_width, channels), dtype=numpy.float32)
    
    inverse_transform_matrix = numpy.linalg.inv(transform_function.args[0])
    
    for y in range(new_height):
        for x in range(new_width):
            
            inverse_transformed_point = numpy.matmul(inverse_transform_matrix, [x + min_x, y + min_y, 1])
            original_x, original_y, _ = inverse_transformed_point
            
            if 0 <= original_x < width - 1 and 0 <= original_y < height - 1:
                x0 = int(original_x)
                x1 = x0 + 1
                y0 = int(original_y)
                y1 = y0 + 1
                for c in range(channels):
                    p00 = image[y0, x0, c]
                    p01 = image[y0, x1, c]
                    p10 = image[y1, x0, c]
                    p11 = image[y1, x1, c]
                    interpolated_pixel_value = bilinear_interpolate(p00, p01, p10, p11, original_x - x0, original_y - y0)
                    result[y, x, c] = interpolated_pixel_value
                    
    print("DONE")
    return result.astype(numpy.uint8)

#1.3a) All 3 functions can be fully tested by using simple examples and checking if the values are correectly inserted into
#the matrix. The expected values for a rotation of 90° are: [[6.123234e-17,1,0],[-1,6.123234e-17,0],[0,0,1]].
#The usage of those matrixes must be tested in combination with transform_image. As i am rescaling the size of the new image,
#it is pretty hard to test translation
def translation( tx, ty ):
    
    return numpy.array([
        [1, 0, tx],
        [0, 1, ty],
        [0, 0, 1]
    ])

def scale( sx, sy ):

    return numpy.array([
        [sx, 0, 0],
        [0, sy, 0],
        [0, 0, 1]
    ])

def rotation( alpha ):
    cos_alpha = numpy.cos(alpha)
    sin_alpha = numpy.sin(alpha)

    return numpy.array([
        [cos_alpha, sin_alpha, 0],
        [-sin_alpha, cos_alpha, 0],
        [0, 0, 1]
    ])
def inv_translation( tx, ty ):
    
    return numpy.array([
        [1, 0, -tx],
        [0, 1, -ty],
        [0, 0, 1]
    ])

def inv_scale( sx, sy ):

    return numpy.array([
        [1/sx, 0, 0],
        [0, 1/sy, 0],
        [0, 0, 1]
    ])

def inv_rotation( alpha ):
    cos_alpha = numpy.cos(alpha)
    sin_alpha = numpy.sin(alpha)

    return numpy.array([
        [cos_alpha, -sin_alpha, 0],
        [sin_alpha, cos_alpha, 0],
        [0, 0, 1]
    ])

def centered_rotation( ox, oy, alpha ):
    cos_alpha = numpy.cos(alpha)
    sin_alpha = numpy.sin(alpha)

    translate_to_origin = numpy.array([
        [1, 0, -ox],
        [0, 1, -oy],
        [0, 0, 1]
    ])

    rotate = numpy.array([
        [cos_alpha, sin_alpha, 0],
        [-sin_alpha, cos_alpha, 0],
        [0, 0, 1]
    ])

    translate_back = numpy.array([
        [1, 0, ox],
        [0, 1, oy],
        [0, 0, 1]
    ])

    transform_matrix = numpy.matmul(translate_back, numpy.matmul(rotate, translate_to_origin))

    return transform_matrix

def some_function( M, x, y ):
 
    # bitte ersetzen Sie diese Funktion, so wie es für den Test benötigt wird
    
    return x,y

def affine_transformaton( M, x, y ):
    point = numpy.array([x, y, 1])
    result = numpy.matmul(M, point)

    return result

def affine_transformaton_function( M ):
    bound_function = partial( affine_transformaton, M )
    return bound_function

def compute_perspective_projection_from_corresponding_points(S, D):
    
    A = []
    for s, d in zip(S, D):
        sx, sy = s
        dx, dy = d
        A.append([sx, sy, 1, 0, 0, 0, -dx * sx, -dx * sy])
        A.append([0, 0, 0, sx, sy, 1, -dy * sx, -dy * sy])
    A = numpy.array(A, dtype=numpy.float32)

    b = []
    for d in D:
        dx, dy = d
        b.extend([dx, dy])
    b = numpy.array(b)
    
    error = []
    for i in range(len(A)):
        error.append(numpy.linalg.norm(b - numpy.dot(A, x)))
    min_index = numpy.argmin(error)

    A_min_index = A[min_index].tolist()
    A_min_index.append(0)
    
    perspective_matrix = numpy.array(A_min_index).reshape(3, 3)
    partial_transform_function = partial(affine_transformaton, perspective_matrix)
    print(perspective_matrix)
    return partial_transform_function

class TestNotebook(unittest.TestCase):
    '''
    def test_bilinear_interpolate(self):
        print(bilinear_interpolate( 1, 0, 1, 0, 0.5, 0.5))
        
    def test_translation(self):
        print(translation(-30,90))
    
    def test_rotation(self):
        print(rotation((90.0 / 360.0) * (numpy.pi * 2.0)))
        
    def test_scale(self):
        print(scale(2,2))

    def test_transform_image(self):
        array = [[1,1,0],[1,1,1],[0,1,1]]
        image = original_image = cv2.imread('kodim15.png').astype( numpy.float32 )        
        transformed_image = transform_image( image, partial(affine_transformaton, array) )
        cv2.imwrite('kodim15_transformed.png', transformed_image) 

    def test_translation_image(self):
        image = original_image = cv2.imread('kodim15.png').astype( numpy.float32 )
        tx = -50
        ty = 22.5
        transform = affine_transformaton_function( translation(tx,ty) )
        transformed_image = transform_image( image, transform )
        cv2.imwrite('translated.png', transformed_image) 

    def test_scale_image(self):
        image = original_image = cv2.imread('kodim15.png').astype( numpy.float32 )
        sx = 1
        sy = -1
        transform = affine_transformaton_function( scale(sx,sy) )
        transformed_image = transform_image( image, transform )
        cv2.imwrite('scaled.png', transformed_image) 

    def test_rotate_image(self):
        image = original_image = cv2.imread('kodim15.png').astype( numpy.float32 )
        alpha = (45.0 / 360.0) * (numpy.pi * 2.0)
        transform = affine_transformaton_function( rotation(alpha) )
        transformed_image = transform_image( image, transform )
        cv2.imwrite('rotated.png', transformed_image)
    
    def test_centered_rotate_image(self):
        image = original_image = cv2.imread('kodim15.png').astype( numpy.float32 )
        alpha = (45.0 / 360.0) * (numpy.pi * 2.0)
        transform = affine_transformaton_function( centered_rotation(100,100,alpha) )
        transformed_image = transform_image( image, transform )
        cv2.imwrite('centered_rotated.png', transformed_image) 
    '''
    def test_perspective_correction(self):
        image = original_image = cv2.imread('perspective.jpg').astype( numpy.float32 )
        P_old = [( 422,199), ( 987,281), (25, 809), ( 917,1040) ]
        P_new = [(   0,  0), (1031,  0), ( 0,1172), (1031,1172) ]
        transform = compute_perspective_projection_from_corresponding_points( P_old, P_new )
        transformed_image = transform_image( image, transform )
        cv2.imwrite('perspective_corrected.jpg', transformed_image) 
 

unittest.main(argv=[''], verbosity=2, exit=False)
#print(centered_rotation(100,100,(45.0 / 360.0) * (numpy.pi * 2.0)))

test_perspective_correction (__main__.TestNotebook) ... ERROR

ERROR: test_perspective_correction (__main__.TestNotebook)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-6cbb171160c5>", line 255, in test_perspective_correction
    transform = compute_perspective_projection_from_corresponding_points( P_old, P_new )
  File "<ipython-input-14-6cbb171160c5>", line 190, in compute_perspective_projection_from_corresponding_points
    error.append(numpy.linalg.norm(b - numpy.dot(A, x)))
NameError: name 'x' is not defined

----------------------------------------------------------------------
Ran 1 test in 0.025s

FAILED (errors=1)
